<a href="https://colab.research.google.com/github/sendyprstyo/Sendy_KC_TI1B/blob/main/Content_Based_Filtering_Spotify_Song.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

# Load Dataset

In [ ]:
# Load dataset
df = pd.read_csv("spotify-2023.csv", encoding='latin1')

In [ ]:
df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5


# Pre-Processing

In [ ]:
# Preprocessing
for col in ['streams', 'in_deezer_playlists', 'in_shazam_charts']:
    df[col] = df[col].astype(str).str.replace(",", "")
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in df.select_dtypes(include='object').columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

for col in df.select_dtypes(include='number').columns:
    df[col].fillna(df[col].median(), inplace=True)

categorical_features = ['artist(s)_name', 'key', 'mode']
numerical_features = [
    'bpm', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%',
    'instrumentalness_%', 'liveness_%', 'speechiness_%'
]

df['text_features'] = df[categorical_features].agg(' '.join, axis=1)

scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(df[numerical_features])
numerical_scaled_df = pd.DataFrame(numerical_scaled, columns=numerical_features)

full_features = pd.concat([df[['track_name', 'text_features']].reset_index(drop=True), numerical_scaled_df], axis=1)

/tmp/ipython-input-7-1477246204.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-7-1477246204.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

# Vectorization

In [ ]:
# TF-IDF vectorization
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(full_features['text_features'])

# Combine TF-IDF and numeric features
final_matrix = hstack([tfidf_matrix, numerical_scaled_df.values])

# Compute similarity
similarity = cosine_similarity(final_matrix)

# Calculate Similarity

In [ ]:
def recommend(track_name, top_n=10):
    track_name_lower = track_name.lower()
    track_names_lower = [t.lower() for t in full_features['track_name'].values]
    if track_name_lower not in track_names_lower:
        return f"Lagu '{track_name}' tidak ditemukan dalam dataset."
    idx = track_names_lower.index(track_name_lower)
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i for i, score in sim_scores[1:top_n+1]]
    recommendations = []
    for i in top_indices:
        recommendations.append({
            'track_name': df.iloc[i]['track_name'],
            'artist(s)_name': df.iloc[i]['artist(s)_name'],
            'similarity_score': round(sim_scores[i][1], 3)
        })
    recommendation_df = pd.DataFrame(recommendations)
    return recommendation_df


# Penggunaan

In [ ]:
if __name__ == "__main__":
    print("Sistem Rekomendasi Lagu - Content Based")
    user_input = input("Masukkan judul lagu: ")
    results = recommend(user_input)
    print("\nRekomendasi:")
    print(results)


Sistem Rekomendasi Lagu - Content Based
Masukkan judul lagu: LALA

Rekomendasi:
                              track_name                      artist(s)_name  \
0                      PLAYA DEL INGLï¿½                Myke Towers, Quevedo   
1                                Sï¿½ï¿½  Anuel Aa, Myke Towers, Jhay Cortez   
2                               El Cielo    Feid, Myke Towers, Sky Rompiendo   
3  That That (prod. & feat. SUGA of BTS)                           PSY, Suga   
4                              The Feels                               TWICE   
5                      Super Freaky Girl                         Nicki Minaj   
6                            A Tu Merced                           Bad Bunny   
7                             DANCE CRIP                              Trueno   
8        There's Nothing Holdin' Me Back                        Shawn Mendes   
9                                  QUEMA        Sog, Ryan Castro, Peso Pluma   

   similarity_score  
0             0.5